In [12]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import xmltodict

In [4]:
def get_all_cols(x: OrderedDict, prefix: str='', page_set: set=None) -> set:
    """
    Iterates through a nested ordered dict and returns a set of columns which would be required
    for each "page" of data, where pages are "delimited" by the element tag "IndexNumberId".
    Proper usage is to leave the optional arguments as their default values. They are used
    internally by the function to pass the same page_set object and the proper prefixes into
    recursive calls. Example data:
    
    OrderedDict([
    
             ('numerical data', '####'),
             ('string data', 'something'),
             
             ('nested OrderedDict',
              OrderedDict([('PrimaryClaimCauseOfAction',
                            OrderedDict([('CauseOfActionType', 'Non-Payment'),
                                         ('Amount', '####')]))])),
                                         
             ('nested OrderedDict with possible duplicate columns',
                  OrderedDict([('Party',
                               [OrderedDict([('Role', 'something else'),
                                             ('PartyType', 'something else'),
                                             ('RepresentationType', 'something else'),
                                             ('Undertenant', 'something else')]),
                                OrderedDict([('Role', 'something else'),
                                             ('PartyType', 'something else'),
                                             ('RepresentationType', 'something else'),
                                             ('Undertenant', 'something else')])])])),
                                             
             ('this is just a list because this data is bad',
              ['list item 1', 'list item 2'])
              
              ])
    
    The set output from this page of data is:
    
    {
    '/numerical data',
    '/string data',
    '/nested OrderedDict/PrimaryClaimCauseOfAction/CauseOfActionType',
    '/nested OrderedDict/PrimaryClaimCauseOfAction/Amount',
    '/nested OrderedDict with possible duplicate columns/Party/Role',
    '/nested OrderedDict with possible duplicate columns/Party/PartyType',
    '/nested OrderedDict with possible duplicate columns/Party/RepresentationType',
    '/nested OrderedDict with possible duplicate columns/Party/Undertenant',
    '/nested OrderedDict with possible duplicate columns/Party/Role_2',
    '/nested OrderedDict with possible duplicate columns/Party/PartyType_2',
    '/nested OrderedDict with possible duplicate columns/Party/RepresentationType_2',
    '/nested OrderedDict with possible duplicate columns/Party/Undertenant_2',
    '/this is just a list because this data is bad',
    }
    
    """
    if page_set is None:
        page_set = set()
    
    for i in x.items():
        if isinstance(i[1], OrderedDict):
            # If the data is an OrderedDict, then the funciton is called recursively until it is
            # not. Along the way, it prepends a prefix to show absolute path within the page.
            get_all_cols(i[1], prefix + '/' + i[0], page_set)
        elif isinstance(i[1], list):
            # If the data is a list (in this case, of OrderedDict objects), then the function is
            # called on all items in the list. The list of OrderedDicts will execute by the block
            # above unless they are themselves lists or simply what is effectively a 2-tuple of
            # strings.
            for item in i[1]:
                # Some data is a simple list of strings. When they get here, only items which
                # have an "items" attribute can be properly iterated by the top level loop. There
                # is a but of duck typing going on here, but it's a risk I'm willing to take.
                if hasattr(item, 'items'):
                    get_all_cols(item, prefix + '/' + i[0], page_set)
        else:
            if prefix + '/' + i[0] in page_set:
                for j in range(2,1000):
                    if prefix + '/' + i[0] + f'_{j}' in page_set:
                        pass
                    else:
                        page_set.add(prefix + '/' + i[0] + f'_{j}')
                        break
            else:
                page_set.add(prefix + '/' + i[0])
    return page_set

In [25]:
def create_data(x: OrderedDict, prefix: str='', page_df: pd.DataFrame=None, all_cols: set=None) -> set:
    """
    Iterates through a possibly nested ordered dict (a "page") and returns the data
    from the page in a dataframe. A set with every possible column needs to be provided,
    hence the all_cols function.
    """
    if page_df is None:
        page_df = pd.DataFrame(np.empty((1,len(all_cols)), dtype=str), columns=all_cols)
    for i in x.items():
        if isinstance(i[1], OrderedDict):
            create_data(i[1], prefix + '/' + i[0], page_df)
        elif isinstance(i[1], list):
            for item in i[1]:
                if hasattr(item, 'items'):
                    create_data(item, prefix + '/' + i[0], page_df)
        else:
            if page_df[prefix + '/' + i[0]][0] != '':
                for j in range(2, 1000):
                    if page_df[prefix + '/' + i[0] + f'_{j}'][0] != '':
                        pass
                    else:
                        page_df[prefix + '/' + i[0] + f'_{j}'][0] = i[1]
                        break
            else:
                page_df[prefix + '/' + i[0]][0] = i[1]
    return page_df

In [2]:
%%time
with open('LandlordTenantExtract.xml', 'r') as f:
    """All of the data is actually stored in Extract/Indexes/Index of the parsed object.
    They are read in as list of (nested) ordered dict objects."""
    actual_data = xmltodict.parse(f.read())['Extract']['Indexes']['Index']

Wall time: 2min 14s


In [5]:
%%time
all_cols = set()
for i in actual_data:
    all_cols = all_cols.union(get_all_cols(i))

Wall time: 30 s


In [184]:
len(actual_data)

252019

In [178]:
len(all_cols)

2076

In [29]:
%%time
df_output = pd.DataFrame(np.empty((len(actual_data),len(all_cols)), dtype=str), columns=all_cols)
i = 0
for page in actual_data:
    if not i % 1000:
        print(i)
    df_output.iloc[i] = create_data(page, all_cols=all_cols).iloc[0]
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [30]:
df_output.to_csv("Landlord_output.csv", sep="|", index=False)

In [ ]:
# One way to add all of the columns you'll care about is to identify a common word and do this.
["/IndexNumberId", "/Court", "/FiledDate", "/PropertyType", "/Classification",
           "/SpecialtyDesignations/SpecialtyDesignationType", "/Status","/DisposedDate",
           "/DateOfJuryDemand", "/PropertyAddresses/PropertyAddress/PostalCode"] + sorted([i for i in all_cols if "/Parties" in i])